# 解读torch.utila.data.Dataloader

## demo

In [1]:
from torch.utils.data import DataLoader, Dataset

# 定义一个简单的数据集
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# 创建数据集和 DataLoader
dataset = MyDataset([1, 2, 3, 4, 5])
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 遍历数据加载器
for batch in dataloader:
    print(batch)

tensor([4, 2])
tensor([1, 5])
tensor([3])


## iterator 迭代器

`迭代器`是一个对象，它能够逐个返回数据，直到数据耗尽

### **可迭代对象（Iterable）**

- 实现了 `__iter__()` 方法，返回一个迭代器。

### **迭代器（Iterator）**

- 实现了 `__iter__()` 和 `__next__()` 方法。

In [2]:
# 定义一个列表，可迭代对象
my_list = [1, 2, 3]
print(my_list)
# 获取列表的迭代器
iterator = iter(my_list)

print(next(iterator))  # 输出 1
print(next(iterator))  # 输出 2
print(next(iterator))  # 输出 3

[1, 2, 3]
1
2
3


In [3]:
print(next(iterator))

StopIteration: 

## torch.utils.data.Dataloader

## sampler

`sampler`是控制数据加载顺序的核心组件，它定义了数据集中的样本如何被选择

- 随机采样（`RandomSampler`）：用于打乱数据以提升模型泛化能力。
- 顺序采样（`SequentialSampler`）：用于按顺序加载数据，例如测试集。
- 自定义采样：用于平衡类别、不均匀数据分布或实现特定采样逻辑。

如果用户不显式指定 `sampler` 参数，`DataLoader` 会根据 `shuffle` 参数选择默认的 `Sampler`:

- **`shuffle=False`**：使用 `SequentialSampler`，按样本索引顺序加载数据。
- **`shuffle=True`**：使用 `RandomSampler`，随机打乱数据。

In [4]:
range(10)

range(0, 10)

In [5]:

from torch.utils.data import Sampler

class MySampler(Sampler):
    def __init__(self, data_source):
        self.data_source = data_source

    def __iter__(self):
        # 返回样本索引的迭代器
        return iter(range(len(self.data_source)))

    def __len__(self):
        return len(self.data_source)

dataset = range(10)
sampler = MySampler(dataset)
print(list(sampler))  # 输出：[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


### SequentialSampler

按顺序采样数据。适合测试集、验证集等不需要随机化的任务

In [6]:
from torch.utils.data import SequentialSampler

sampler = SequentialSampler(range(10))
print(list(sampler))  # 输出：[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


### RandomSampler

随机打乱数据顺序，适合训练集。

In [7]:

from torch.utils.data import RandomSampler

sampler = RandomSampler(range(10))
print(list(sampler))  # 输出示例：[7, 2, 5, 8, 3, 6, 1, 0, 9, 4]


[9, 0, 7, 8, 2, 5, 4, 3, 1, 6]


In [ ]:
sampler

## collate_fn

用于定义如何将单个样本组合成一个批次。它的作用在于将多个样本（通常是单个数据点）聚合成一个可以输入到模型中的批次。不同的任务和数据类型（如图像、文本、序列等）可能需要不同的 collate_fn。

### 默认的collate_fn

PyTorch 默认的 `collate_fn` 主要处理以下几种情况：

- **数值型数据**（如整数或浮点数）：直接堆叠成一个张量（`torch.Tensor`）。
- **字典或列表**：如果样本本身是字典或列表，它会将字典或列表的每个元素进行堆叠。

例如，若数据集中的每个样本都是一个数值，默认的 `collate_fn` 会将它们堆叠成一个 `Tensor`：

In [8]:

from torch.utils.data import DataLoader

data = [1, 2, 3, 4]
dataloader = DataLoader(data, batch_size=2)

for batch in dataloader:
    print(batch)



tensor([1, 2])
tensor([3, 4])


### 自定义 collate_fn

- **变长序列**（如文本、时间序列等），需要填充（padding）到相同的长度。
- **图像和标签**，可能需要同时对图像和标签进行特定的处理

----

* 变长序列

In [9]:

import torch
from torch.utils.data import DataLoader

def collate_fn(batch):
    # 假设 batch 是 [(text1), (text2), ...]，每个 text 是一个不同长度的序列
    # 填充文本到同一长度
    texts, lengths = zip(*batch)
    padded_texts = torch.nn.utils.rnn.pad_sequence(texts, batch_first=True, padding_value=0)
    return padded_texts, torch.tensor(lengths)

# 示例数据：[(句子1，长度1), (句子2，长度2), ...]
data = [(torch.tensor([1, 2]), 2), (torch.tensor([3, 4, 5]), 3)]
dataloader = DataLoader(data, batch_size=2, collate_fn=collate_fn)

for batch in dataloader:
    padded_texts, lengths = batch
    print(padded_texts)
    print(lengths)



tensor([[1, 2, 0],
        [3, 4, 5]])
tensor([2, 3])


* 图像和标签的组合

In [10]:
import torch
from torch.utils.data import DataLoader
from PIL import Image
from torchvision import transforms

def collate_fn(batch):
    images, labels = zip(*batch)
    # 将图像转为张量并堆叠
    transform = transforms.ToTensor()
    images = torch.stack([transform(image).float() for image in images])
    # 将标签堆叠成一个张量
    labels = torch.tensor(labels)
    return images, labels

# 示例数据：[(图像1, 标签1), (图像2, 标签2), ...]
data = [(Image.new('RGB', (100, 100), color='red'), 0),
        (Image.new('RGB', (100, 100), color='blue'), 1)]
dataloader = DataLoader(data, batch_size=2, collate_fn=collate_fn)

for batch in dataloader:
    images, labels = batch
    print(images.shape)  # 输出：torch.Size([2, 100, 100, 3])
    print(labels)  # 输出：tensor([0, 1])

torch.Size([2, 3, 100, 100])
tensor([0, 1])
